# Analysis Development
 - Given the user has access to Spotify, this script will return a simple plot of their liked songs grouped by day and number of songs liked per day
 - Additional updated research will be appended in here

##### Import the modules

In [26]:
import numpy as np
import pandas as pd

import spotipy
from spotipy import SpotifyOAuth

##### Client ID and Secret are passed from an external source

In [27]:
import settings

In [28]:
# Client ID and Client Secret is called and used
spotify_client_id = settings.SPOTIFY_CLIENT_ID
spotify_client_secret = settings.SPOTIFY_CLIENT_SECRET
spotify_redirect_uri = "http://localhost:5000/callback"

##### With the application setup, we just need to call in the authenticator and method itself
 - This will vary based on the type of access. For me, it is suitable.

In [29]:
# provide scope of access for Python which requests access from the current user
scope =  'user-library-read user-top-read user-read-private playlist-read-private playlist-read-collaborative playlist-modify-public playlist-modify-private'

# authentication manager ensures secured point of access from spotify to python to client (me)
auth_manager = SpotifyOAuth(
                            scope = scope, 
                            client_id = spotify_client_id,
                            client_secret = spotify_client_secret,
                            redirect_uri = spotify_redirect_uri
                            )

# initial call of spotify using authentication manager
sp = spotipy.Spotify(auth_manager = auth_manager)

##### Test connection (expectancy of a dict response with personal info)

In [30]:
sp.current_user()

{'display_name': '121631289',
 'external_urls': {'spotify': 'https://open.spotify.com/user/121631289'},
 'href': 'https://api.spotify.com/v1/users/121631289',
 'id': '121631289',
 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b82d01c89064802f0d7d779da0f',
   'height': 64,
   'width': 64},
  {'url': 'https://i.scdn.co/image/ab6775700000ee85d01c89064802f0d7d779da0f',
   'height': 300,
   'width': 300}],
 'type': 'user',
 'uri': 'spotify:user:121631289',
 'followers': {'href': None, 'total': 13},
 'country': 'US',
 'product': 'premium',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'email': 'nsung214@gmail.com'}

##### Users Top Tracks

In [31]:
sp.current_user_top_tracks()

{'items': [{'album': {'album_type': 'SINGLE',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4iMO20EPodreIaEl8qW66y'},
      'href': 'https://api.spotify.com/v1/artists/4iMO20EPodreIaEl8qW66y',
      'id': '4iMO20EPodreIaEl8qW66y',
      'name': 'Still Woozy',
      'type': 'artist',
      'uri': 'spotify:artist:4iMO20EPodreIaEl8qW66y'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',
     'SK',
     'ES',
     'SE',
     'CH',
     'TW',
     'TR',
     'UY',
     'US',
     'GB',
 

##### While loop to collect all songs in my liked

In [32]:
liked_songs = []
offset = 0
users_liked = sp.current_user_saved_tracks(limit = 50)

while users_liked["next"] != None:
    liked_songs.extend(users_liked["items"])
    offset+=50
    users_liked = sp.current_user_saved_tracks(offset=offset,limit = 50)

liked_songs.extend(users_liked["items"])

##### Observed collection of my liked songs 

In [33]:
len(liked_songs)

1796

In [34]:
df = pd.DataFrame(list(map(lambda x: x["track"],liked_songs)))

In [35]:
df2 = pd.DataFrame(liked_songs)

In [36]:
df["added_at"] = pd.to_datetime(df2["added_at"]).dt.strftime('%Y-%m-%dT%H:%M:%SZ')


In [37]:
# df["release_date"] = df["album"].apply(lambda x: x["release_date"])
df["release_date"] = pd.to_datetime(df["album"].apply(lambda x: x["release_date"]), format='ISO8601').dt.strftime('%Y-%m-%d')

In [39]:
from datetime import datetime, timezone
df["added_at"] = pd.to_datetime(df["added_at"], format= 'ISO8601')
df["release_date"] = pd.to_datetime(df["release_date"], format= 'ISO8601')

In [42]:
df["added_at"] = pd.to_datetime(df["added_at"].dt.strftime('%Y-%m-%d'))

In [49]:
df["date_difference"] = (df["added_at"]-df["release_date"]).dt.days

In [58]:
# df["added_at"].dt.day
# df["added_at"].dt.month
# df["added_at"].dt.year

df2 = df["added_at"].dt.month
df2.groupby("added_at").mean

KeyError: 'added_at'